In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from Levenshtein import distance as levenshtein_distance
import faiss

In [ ]:
def read_data():
    employees = pd.read_parquet('employees.parquet')
    orcs = pd.read_parquet('orcs.parquet')
    return employees, orcs

def preprocess_dates(df):
    dates = pd.to_datetime(df['birthdate'], errors='coerce')
    df['year'] = dates.dt.year
    df['month'] = dates.dt.month
    df['day'] = dates.dt.day

    for col in ['year', 'month', 'day']:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

    return df

def build_string_vectorizer():
    return TfidfVectorizer(analyzer='char_wb', ngram_range=(2,4))

def vectorize_strings(df, fields, vectorizer=None):
    df[fields] = df[fields].fillna('')
    text = df[fields].apply(lambda x: ' '.join(x), axis=1)

    if vectorizer is None:
        vectorizer = build_string_vectorizer()
        X = vectorizer.fit_transform(text)
        return X, vectorizer
    else:
        X = vectorizer.transform(text)
        return X

def encode_dates(df, enc=None, scaler=None):
    if enc is None:
        enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        month_ohe = enc.fit_transform(df[['month']])
    else:
        month_ohe = enc.transform(df[['month']])

    numeric_feats = np.vstack([
        df['day'].values,
        df['year'].values
    ]).T.astype(float)

    if scaler is None:
        scaler = StandardScaler()
        numeric_feats = scaler.fit_transform(numeric_feats)
    else:
        numeric_feats = scaler.transform(numeric_feats)

    return np.hstack([numeric_feats, month_ohe]), enc, scaler

def build_ivf_index(embeddings, nlist=100):
    # использовал IVF индекс так как он ускоряет поиск близких эмбеддингов, масштабируется под большие данные и удобен в настройке
    dimension = embeddings.shape[1]
    quantizer = faiss.IndexFlatL2(dimension)
    index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
    index.train(embeddings)
    index.add(embeddings)
    return index

def compute_metric(emp_row, orc_row):
    #Использовал кастомную комбинированную метрику, так как у нас даны строковые и числовые данные одновременно
    #Для числовых данных евклидовая метрика для строковых расстояние Левентшейна
    str_fields = ['name', 'surname', 'fathername', 'inn', 'passport']
    str_distance = 0
    for f in str_fields:
        str_distance += levenshtein_distance(str(emp_row[f]), str(orc_row[f]))

    day_diff = emp_row['day'] - orc_row['day']
    year_diff = emp_row['year'] - orc_row['year']
    month_diff = 1 if emp_row['month'] != orc_row['month'] else 0
    num_distance = np.sqrt(day_diff**2 + year_diff**2 + month_diff**2)

    return str_distance + num_distance


In [ ]:
employees, orcs = read_data()
employees = preprocess_dates(employees)
orcs = preprocess_dates(orcs)

str_fields = ['name', 'surname', 'fathername', 'inn', 'passport']

X_emp_str, str_vectorizer = vectorize_strings(employees, str_fields)
X_orc_str = vectorize_strings(orcs, str_fields, vectorizer=str_vectorizer)

svd = TruncatedSVD(n_components=100, random_state=42)
X_emp_str_reduced = svd.fit_transform(X_emp_str)
X_orc_str_reduced = svd.transform(X_orc_str)

emp_num_feats, enc, scaler = encode_dates(employees)
orc_num_feats, _, _ = encode_dates(orcs, enc=enc, scaler=scaler)

X_emp = np.hstack([X_emp_str_reduced, emp_num_feats]).astype('float32')
X_orc = np.hstack([X_orc_str_reduced, orc_num_feats]).astype('float32')


In [ ]:
mask_m = (employees['gender'] == 'м')
X_m = X_emp[mask_m]
emp_indices_m = employees.index[mask_m].to_numpy()
index_m = build_ivf_index(X_m)

mask_f = (employees['gender'] == 'ж')
X_f = X_emp[mask_f]
emp_indices_f = employees.index[mask_f].to_numpy()
index_f = build_ivf_index(X_f)

indexes_by_gender = {
    'м': index_m,
    'ж': index_f
}

emp_indices_by_gender = {
    'м': emp_indices_m,
    'ж': emp_indices_f
}


In [ ]:
k = 2
threshold = 14  # лучшие значения подобранные методом проб и ошибок

orc_candidates = []

for i, orc_row in orcs.iterrows():
    g = orc_row['gender']
    if g not in indexes_by_gender:
        continue

    index_g = indexes_by_gender[g]
    emp_indices = emp_indices_by_gender[g]

    x_orc_vec = X_orc[i:i+1]

    D, I = index_g.search(x_orc_vec, k)
    neighbor_idx = I[0]
    candidate_ids = emp_indices[neighbor_idx]

    for emp_id in candidate_ids:
        emp_row = employees.loc[emp_id]
        m = compute_metric(emp_row, orc_row)
        if m < threshold:
            orc_candidates.append(emp_id)

orc_candidates = np.unique(orc_candidates).astype(np.uint64)

res = pd.DataFrame({
    'orig_index': orc_candidates
}).reset_index(names='id')

res.to_parquet('submission.parquet', index=False)


In [ ]:
print("Потенциальные орки среди сотрудников:", orc_candidates)
print("Количество людей, признанных орками:", len(orc_candidates))


Потенциальные орки среди сотрудников: [      3     160     400 ... 1011543 1011585 1011658]
Количество людей, признанных орками: 10530


 Анализ работы: итоговый скор 0.751 приведен чистовой вариант работы без лишних аутпутов
 предобработка дат заменяем наны на среднее по столбцу после чего день и год кодируются числами, а к месяцу применяем метод OneHotEncoder
 для строк заменяем наны на пустые строчки (возможно стоило заменять на что-то другое, но в ходе работы было выявлено, что это ухудшает модель)
 далее строки кодировались в векторы с помощью TF-IDF(пришлось использовать чтобы работать с faiss (в чате писали про этот метод)), после чего размер веткторов уменьшался с помощью SVD
 По скольку пол единственное место где нет опечаток, то было принято решение строить индексы для двух полов отдельно, это улучшило модель